In [66]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import math
import sys

%matplotlib qt

def dist(p1, p2): # Calculates the distance between two points in three-dimensional space.
    return math.sqrt((p1[0]-p2[0]) ** 2 + (p1[1]-p2[1]) ** 2 + (p1[2]-p2[2]) ** 2)+0.00000000001

def rgbToHex(r, g, b): # Takes a given series of numbers and converts them to a hexadecimal string.
    rStr = str(hex(r))[2:]
    if len(rStr) == 1:
        rStr = "0" + rStr
    gStr = str(hex(g))[2:]
    if len(gStr) == 1:
        gStr = "0" + gStr
    bStr = str(hex(b))[2:]
    if len(bStr) == 1:
        bStr = "0" + bStr
    return "#" + rStr + gStr + bStr

class potVal: # Class for points on the graph, storing x-y-z coordinate data, the index of its equipotential surface (if any), and its potential value from the points around it.
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        totPot = 0
        for n in range(0, len(charges)):
            totPot += ((8.98e9)*(charges[n]))/(distOfUnit*dist([posX[n],posY[n],posZ[n]],[x,y,z]))
        self.val = totPot
        self.eqLine = -1

# LOAD FROM FILE - Asks the user if they wish to load data from a save file.
loadYN = input("(Y/N) Would you like to load a distribution from a file? (Leave blank for no.)").lower()
if loadYN == "yes" or loadYN == "y":
    loadYN = True
else:
    loadYN = False

xValsP = []
yValsP = []
zValsP = []
voltVals = []
neg = False
pos = False
if loadYN:
    fileName = input("Please enter the name of the file you would like to load from. (Leave blank for the default, save.dat.)")
    if fileName == "":
        fileName = "save.dat"
    with open(fileName, 'r') as f:
        numSurfs = int(f.readline())
        if int(f.readline()) == 1:
            voltVals.append("")
            pos = True
        for n in range(0, numSurfs):
            xValsP.append([])
            yValsP.append([])
            zValsP.append([])
            voltVals.append("")
        for line in f:
            if line[0] == "#":
                x = float(line[1:line.index("x")])
                y = float(line[line.index("x") + 1:line.index("y")])
                z = float(line[line.index("y") + 1:line.index("z")])
                v = line[line.index("z") + 1:line.index("v")]
                n = int(line[line.index("v") + 1:])
                if float(v) < 0:
                    neg = True
                else:
                    pos = True
                ind = n
                if pos:
                    ind -= 1
                xValsP[ind].append(x)
                yValsP[ind].append(y)
                zValsP[ind].append(z)
                voltVals[n] = v
else:
    # SCALE - Defines physical length of the axes. 
    scale = input("What scale factor would you like for the axes? (Leave blank for the default, 1, which is 1.4 nm.)")
    if scale == "":
        scale = 1
    else:
        scale = int(scale)
    axisLength = scale * 1.4e-9

    # NUMBER OF CHARGES - Defines the number of charged objects to consider.
    numCharges = input("How many charges would you like to model? (Leave blank for the default, which is 1.)")
    if numCharges == "":
        numCharges = 1
    else:
        numCharges = int(numCharges)

    # CHARGE VALUES - Defines the charge values in C for each charged object.
    chargesUnscaled = []
    charges = []
    manualChargeInput = input("(Y/N) Would you like to manually enter the charge value(s)? (Enter 1 for a charge of 1.6e-19 C.) (Enter N for the default, which is all electrons.)").lower()
    if manualChargeInput == "yes" or manualChargeInput == "y":
        for n in range(0, numCharges):
            c = input("Please enter the value for charge " + str(n+1) + ".")
            chargesUnscaled.append(int(c))
    else:
        for n in range(0,numCharges):
            chargesUnscaled.append(-1)
    for c in chargesUnscaled:
        charges.append(c*1.6e-19)
        if c < 0:
            neg = True
        else:
            pos = True

    # CHARGE POSITIONS - Defines the x-y-z coordinate values for each charge.
    posX = []
    posY = []
    posZ = []
    distOfUnit = axisLength/100
    for n in range(0,len(charges)):
        posStr = input(("Please enter the coordinates for charge " + str(n+1) + " with charge " + str(charges[n]) + ".")).strip("() ").split(",")
        posX.append(int(posStr[0]))
        posY.append(int(posStr[1]))
        posZ.append(int(posStr[2]))

    # NUMBER OF EQUIPOTENTIAL SURFACES - Defines the number of equipotential surfaces to be used.
    numSurfs = input("How many equipotential surfaces would you like to model? (Leave blank for the default, 10.)")
    if numSurfs == "":
        numSurfs = 10
    else:
        numSurfs = abs(int(numSurfs))
    eqDiff = scale
    eqDiffArr = []
    eqDiffRange = []
    if neg:
        for n in range(-int(numSurfs/(eqDiff/scale)), 0):
            eqDiffArr.append(n*eqDiff)
    if pos:
        for n in range(1, int(numSurfs/(eqDiff/scale)) + 1):
            eqDiffArr.append(n*eqDiff)

    # SAVE TO FILE - Saves the generated distribution to a file. (Actual code doing this is later.)
    saveYN = input("(Y/N) Would you like to save this distribution to a file? (Leave blank for no.)").lower()
    saveFileName = ""
    if saveYN == "y" or saveYN == "yes":
        saveYN = True
        saveFileName = input("Please enter the name of the file you would like to save to. (Leave blank for the default, save.dat.)")
        if saveFileName == "":
            saveFileName = "save.dat"
    else:
        saveYN = False

    potVals = []
    accuracy = 1 # Beware increasing this value much. Even just 2 makes it take significantly longer, but the results are only marginally better at best, so it's not really worth increasing.
    its = 0
    progress = "\rRunning...⠀[" + " "*25 + "]"
    for x in range(int(-100*accuracy), int(100*accuracy)): # This series of nested for loops creates a potVal object for each point and adds it to an array. The bulk of the calculations happen here.
        sys.stdout.write("\r" + progress)
        for y in range(int(-100*accuracy), int(100*accuracy)):
            pVold = potVal(0, 0, 0)
            pVold.val = 0
            for z in range(int(-100*accuracy), int(100*accuracy)):
                its += 1
                pV = potVal(x/accuracy, y/accuracy, z/accuracy)
                for n in eqDiffArr:
                    if (n < pV.val and n > pVold.val) or (n > pV.val and n < pVold.val):
                        pV.eqLine = int(n/eqDiff)
                        potVals.append(pV)
                pVold = pV
        if (x + 100*accuracy)%((2*100*accuracy)/25) == 0:
            progress = progress.replace(" ", "█", 1)
    print("\nDone!\nGenerating graph...")

    xVals = []
    yVals = []
    zVals = []
    if pos:
        eqDiffArr.insert(eqDiffArr.index(1) - 1, 0)
    for n in eqDiffArr: # Creates an array of arrays of x, y, and z for each equipotential surface.
        xVals.append([])
        yVals.append([])
        zVals.append([])
        voltVals.append("problematic volt val index")
    for p in potVals: # Adds the coordinates for each point on each surface to the above arrays.
        ind = p.eqLine
        if neg:
            ind += numSurfs
        xVals[ind].append(p.x*(1e9*distOfUnit))
        yVals[ind].append(p.y*(1e9*distOfUnit))
        zVals[ind].append(p.z*(1e9*distOfUnit))
        valStr = str(p.val)
        voltVals[p.eqLine] = valStr[0:valStr.find(".")+4]

    for n in range(0,len(xVals)): # Trims arrays.
        if len(xVals[n]) != 0:
            xValsP.append(xVals[n])
            yValsP.append(yVals[n])
            zValsP.append(zVals[n])

    if saveYN: # Actual code for creating the save file.
        with open(saveFileName, 'w') as f:
            f.write(str(len(xValsP)) + "\n")
            if pos:
                f.write(str(1) + "\n")
            else:
                f.write(str(0) + "\n")
            for n in range(0, len(xValsP)):
                for xyz in range(0, len(xValsP[n])):
                    f.write("#")
                    ind = n
                    if pos:
                        ind += 1
                    f.write(str(xValsP[n][xyz]) + "x")
                    f.write(str(yValsP[n][xyz]) + "y")
                    f.write(str(zValsP[n][xyz]) + "z")
                    f.write(voltVals[ind] + "v")
                    f.write(str(ind) + "\n")

fig = plt.figure()
plt.get_current_fig_manager().window.showMaximized()
ax = plt.axes(projection = '3d')
print(len(voltVals))
print(pos)
print(neg)
for n in range(0, len(xValsP)): # Each iteration plots a new surface.
    frac = float(n)/float(len(xValsP)) + 0.1
    r = int(-127*math.cos(2*math.pi*frac)) + 127
    g = int(-127*math.cos(math.pi*frac)) + 127
    b = int(150*math.cos(1.75*(0.5*math.pi*frac + 0.25*math.pi))) + 150
    c = rgbToHex(r, g, b)
    i = n
    if neg and not pos:
        i = len(xValsP) - 1 - n
    elif (neg and pos) and n < 10:
        i = len(xValsP) - 11 - n
    j = n
    if pos:
        j += 1
    if float(voltVals[j]) < 0:
        j = len(xValsP) - j - 10
        if neg and not pos:
            j -= 1
    print(i)
    print(len(xValsP[i]))
    ax.plot(xValsP[i], yValsP[i], zValsP[i], 'o', label = voltVals[j]+"V", color = c)
ax.legend()
plt.tight_layout()
ax.set_xlabel('nm')
ax.set_ylabel('nm')
ax.set_zlabel('nm')

10
False
True
9
63253
8
16570
7
7394
6
4154
5
2642
4
1850
3
1346
2
1034
1
842
0
666


Text(0.5, 0, 'nm')